In [1]:
#from common.util.date import get_current_date

import os
from os import linesep
from dotenv import load_dotenv
import pandas as pd
from datetime import date,datetime,timedelta
import time           

from genai.credentials import Credentials
from genai.schemas import GenerateParams
from genai.model import Model

load_dotenv()

ModuleNotFoundError: No module named 'dotenv'

In [2]:
# api_key = os.getenv("GENAI_KEY", None)
# api_endpoint = os.getenv("GENAI_API", None)
api_key = "pak-u86dLJFt--vsZCir36bDB74iKatSEBaYYzsJPk36_eM"
api_endpoint = "https://bam-api.res.ibm.com/v1"
creds = Credentials(api_key, api_endpoint)

In [3]:
model_name = 'codellama/codellama-34b-instruct'

In [4]:
params = GenerateParams(decoding_method="greedy", min_new_tokens=10,max_new_tokens=1200,  stream = False, stop_sequences = ['Done:"Stop here"'],temperature =0.2)
watsonx_llm = Model(model_name, params=params, credentials=creds)

In [5]:
def get_ipsa_prompt():
    
    JSON_PROMPT = """Given an input question and input data dictionary, create a syntactically correct JS code for visualization.
    
    Abbreviations:                   
    CQ - current quarter
    CY - Current Year
    NY - Next Year
    qtr - quarter
    qty - quantity
    
    Rules:
    Please follow the steps below to generate the JS code.
    Step 1. Generate the chart type, which includes ['bar chart', 'line chart', 'pie chart'].
    Step 2. Generate the title for the chart, which is a summary for the input question.
    Step 3. Transform the input data to expected format based on chart type. For example,
    if chart type is 'pie chart':
    input data={{'columns':['Forecasted Quantity'],
      'index':['Seagate','Toshiba','WD'],
      'data':[25612,9488,14145]}}
    transform to:    
    series: [
        {{
            name: 'Forecasted Quantity',
            colorByPoint: true,
            data: [
                {{
                    name: 'Seagate',
                    y: 25612
                }},
                {{
                    name: 'Toshiba',
                    y: 9488
                }},
                {{
                    name: 'WD',
                    y: 14145
                }}
            ]
        }}
    ]
    if chart type is 'bar chart' or 'line chart':
    input data={{'columns':['Rebate Target','Qty Achieved'],
      'index':['2310.1R','2310.2F','2311.1F'],
      'data':[[20000,168000],[20000,664],[15000,2704]]}}
    transform to:    
    series: [
        {{
            name: 'Rebate Target',
            data: [['202310.1F',20000],['202310.2F',20000],['202311.2F',15500]]
        }},
        {{
            name: 'Qty Achieved',
            data: [['202310.1F',16800],['202310.2F',664],['202311.2F',2704]]
        }}
    ]
 
    
    Format the generated output as follows.
    Question:
    Data:
    Type:
    Title: 
    series:    
    Done:"Stop here" 
    
    Examples:    

    Example 1 -
    Question: In the current SOP cycle, for HDD forecasted volume, what is the suppliers mixed 
    Data:{{'columns':['Forecasted Quantity'],
      'index':['Seagate','Toshiba','WD'],
      'data':[25612,9488,14145]}}
    Type: pie chart
    Title: 'HDD Forecasted Volume by Suppliers'
    series: [
        {{
            name: 'Forecasted Quantity',
            colorByPoint: true,
            data: [
                {{
                    name: 'Seagate',
                    y: 25612
                }},
                {{
                    name: 'Toshiba',
                    y: 9488
                }},
                {{
                    name: 'WD',
                    y: 14145
                }}
            ]
        }}
    ]
    Done:"Stop here"
    
    Example 2 -
    Question: For Seagate Show me qty achieved against the rebate target for current qtr to date
    Data:{{'columns':['Rebate Target','Qty Achieved'],
      'index':['2310.1R','2310.2F','2311.1F'],
      'data':[[20000,168000],[20000,664],[15000,2704]]}}
    Type: bar chart
    Title: 'Seagate Quantity Achieved vs Rebate Target'
    xAxis: 'Time'
    yAxis: 'Quantity'
    series: [
        {{
            name: 'Rebate Target',
            data: [['202310.1F',20000],['202310.2F',20000],['202311.2F',15500]]
        }},
        {{
            name: 'Qty Achieved',
            data: [['202310.1F',16800],['202310.2F',664],['202311.2F',2704]]
        }}
    ]
    Done:"Stop here"

    Example 3 -
    Question: Pricing trend for part number 00VN521 & 00VN571
    Data:{{'columns':['00VN521','00VN571'],
      'index':['2306.1F','2307.1R','2307.2F','2308.1F','2310.1R','2311.1F'],
      'data':[[150,187],[150,187],[150,216.96],[150,216.96],[150,201.96],[150,201.96]]}}
    Type: line chart
    Title: 'Pricing Trend for Part Number 00VN521 & 00VN571'
    xAxis: 'Time'
    yAxis: 'Quantity'
    series: [
        {{
            name: '00VN521',
            data: [['2306.1F',150],['2307.1R',150],['2307.2F',150],['2308.1F',150],['2310.1R',150],['2311.1F',150]]
        }},
        {{
            name: '00VN571',
            data: [['2306.1F',187],['2307.1R',187],['2307.2F',216.96],['2308.1F',216.96],['2310.1R',201.96],['2311.1F',201.96]]
        }}
    ]
    Done:"Stop here"

    Input:
    Question: {question}
    Data: {{'columns':{columns},'index':{index},'data':{data}}}

    """
    
    return JSON_PROMPT

In [6]:
def piechart(gen_out):
    jscode = """Highcharts.chart('container', {{
    chart: {{
        type: 'pie'
    }},
    title: {{
        text: {title}
    }},
    tooltip: {{
        valueSuffix: '%'
    }},
    plotOptions: {{
        series: {{
            allowPointSelect: true,
            cursor: 'pointer',
            dataLabels: [{{
                enabled: true,
                distance: 20
            }}, {{
                enabled: true,
                distance: -40,
                format: '{{point.percentage:.1f}}%',
                style: {{
                    fontSize: '1.2em',
                    textOutline: 'none',
                    opacity: 0.7
                }},
                filter: {{
                    operator: '>',
                    property: 'percentage',
                    value: 10
                }}
            }}]
        }}
    }},
    series: {data}
    }});"""    
    jscode = jscode.format(**gen_out)
    return jscode 

In [7]:
def barchart(gen_out):
    jscode = """Highcharts.chart('container', {{
    chart: {{
        type: 'column'
    }},
    title: {{
        text: {title},
        align: 'left'
    }},
    xAxis: {{
        categories: true,
        crosshair: true,
        accessibility: {{
            description: {Xaxis}
        }}
    }},
    yAxis: {{
        min: 0,
        title: {{
            text: {Yaxis}
        }}
    }},
    tooltip: {{
        valueSuffix: true
    }},
    plotOptions: {{
        column: {{
            pointPadding: 0.2,
            borderWidth: 0
        }}
    }},
    series: {data}
    }});"""    
    jscode = jscode.format(**gen_out)
    return jscode 

In [8]:
def linechart(gen_out):
    jscode = """Highcharts.chart('container', {{
    chart: {{
        type: 'line'
    }},
    title: {{
        text: {title},
        align: 'left'
    }},
    xAxis: {{
        categories: true,
        crosshair: true,
        accessibility: {{
            description: {Xaxis}
        }}
    }},
    yAxis: {{
        min: 0,
        title: {{
            text: {Yaxis}
        }}
    }},
    tooltip: {{
        valueSuffix: true
    }},
    plotOptions: {{
        column: {{
            pointPadding: 0.2,
            borderWidth: 0
        }}
    }},
    series: {data}
    }});"""    
    jscode = jscode.format(**gen_out)
    return jscode 

In [44]:
def run_llm(question,data): 
    
    start = datetime.now()   
    prompt=''
    prompt = get_ipsa_prompt()
    prompt = prompt.format(**data)
    print("Prompt:", prompt)
    prompts = [prompt]
    responses = [response.generated_text for response in watsonx_llm.generate(prompts)]
    answer = responses[0].replace ("\n", linesep)
    print(answer)
    if "Type:" in answer:
        type=answer.split('Type:',2)[1].split('Title:',2)[0]
    if type==' bar chart\n    ' or type==' line chart\n    ':
        if "Title:" in answer:
            title = answer.split("Title:",2)[1].split("xAxis:",2)[0]
        if "xAxis:" in answer:
            Xaxis = answer.split("xAxis:",2)[1].split('yAxis:',2)[0]
        if "yAxis:" in answer:
            Yaxis = answer.split("yAxis:",2)[1].split('series:',2)[0] 
        if "series:" in answer:
            series=answer.split("series:",2)[1].split('Done:"Stop here"', 1)[0]
        gen_output = {'type':type, 'title':title, 'Xaxis':Xaxis, 'Yaxis':Yaxis, 'data':series}
    if type==' pie chart\n    ':
        if "Title:" in answer:
            title = answer.split("Title:", 2)[1].split('series',2)[0] 
        if "series:" in answer:
            series=answer.split('series:', 2)[1].split('Done:"Stop here"',2)[0]
        gen_output = {'type':type,'title':title, 'data':series} 
    end = datetime.now()
    print(f'Total time of execution {end - start}')
    return gen_output

In [46]:
if __name__ == '__main__':
    question = "The trend for forecasted qty by Suppliers for HDD "
    data={'question':question,
          'columns':['Seagate','Toshiba','WD'],
          'index':['2306.1R','2307.2F','2308.1R','2309.1R','2310.2F','2311.1F'],
          'data':[[49500,34232,23000],[42893,33940,21293],[57839,24859,29400],[53789,38490,18390],[51627,34859,19403],[56270,39280,28192]]}
    gen_output = run_llm(question,data)
    if gen_output['type']==' pie chart\n    ':
      js = piechart(gen_output)
    if gen_output['type']==' bar chart\n    ':
      js=barchart(gen_output)
    if gen_output['type']==' line chart\n    ':
      js=linechart(gen_output)
    print(js)

Prompt: Given an input question and input data dictionary, create a syntactically correct JS code for visualization.
    
    Abbreviations:                   
    CQ - current quarter
    CY - Current Year
    NY - Next Year
    qtr - quarter
    qty - quantity
    
    Rules:
    Please follow the steps below to generate the JS code.
    Step 1. Generate the chart type, which includes ['bar chart', 'line chart', 'pie chart'].
    Step 2. Generate the title for the chart, which is a summary for the input question.
    Step 3. Transform the input data to expected format based on chart type. For example,
    if chart type is 'pie chart':
    input data={'columns':['Forecasted Quantity'],
      'index':['Seagate','Toshiba','WD'],
      'data':[25612,9488,14145]}
    transform to:    
    series: [
        {
            name: 'Forecasted Quantity',
            colorByPoint: true,
            data: [
                {
                    name: 'Seagate',
                    y: 25612
      